<a href="https://colab.research.google.com/github/rohit2000123sahu/Civil-Runrest-Prediction/blob/main/Twitter_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install twython

In [ ]:
pip install "dask[dataframe]"

     |████████████████████████████████| 136 kB 24.9 MB/s 


In [ ]:
from twython import Twython
import pandas as pd
import numpy as np
import datetime as dt
import time
import dask.dataframe as dd
from dask.multiprocessing import get

In [ ]:
APP_KEY = 'wb5Jje4pW2Wo7mvj4D1ObfaWF'
APP_SECRET = "QZnFPPeWdTIGoI5EUKrPy7FDeQCbZkjl4fDqeGbOQY9BvzRM87"

twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [ ]:
# Locations used to query tweets

locationsTemp=["chennai","delhi","india","ahmedabad","gujarat","tamil","jammu","srinagar","imphal","lucknow",
                "bengaluru","bangalore","bathinda","chandigarh","ludhiana","amritsar","punjab","arunachal","assam",
                "kolkata","westbengal","kerala","karnataka","andhra","telangana","hyderabad","madhya","uttar",
                "maharashtra","haryana",'Jammu','Srinagar','Delhi-NewDelhi','Bathinda','Dehradun','Chandigarh',
                'Ludhiana','Amritsar','Imphal','Lucknow','Jalandhar','Kolkata','Guwahati','Chennai','Patiala',
                'Bhubaneswar','Bengaluru','Patna','Jaipur','Coimbatore','Hyderabad','Ranchi','Thiruvananthapuram',
                'Shimla','Sangrur','Ahmedabad','Karnal','Pulwama','Puducherry','Gurgaon','Agartala','Madurai',
                'Tiruchirappalli','Bangkok','Salem']

locations = set()
for location in locationsTemp:
    locations.add(location.lower())
locations = pd.DataFrame(list(locations))
locations.columns = ["Locations"]
locations.to_csv("Locations of Interest.csv")


In [ ]:
# Keywords used to query tweets

keywords = ['protest','protests','riot','riots','violence','unrest','clash','bandh','issue','rally','demonstration','election','polls','attac']
keywords = pd.DataFrame(keywords)
keywords.columns = ["Keywords"]
keywords.to_csv("Keywords.csv")

In [ ]:
locations = pd.read_csv("Locations of Interest.csv", index_col=0)
keywords = pd.read_csv("Keywords.csv", index_col=0)

In [ ]:
keywordsQuery = ''.join(map(str, [keyword+" OR " for keyword in list(keywords['Keywords'])]))[:-4]
start = 0
to_return = []
loc_list =locations["Locations"].values.tolist()
len_loc = len(loc_list) // 5 + 1
for i in range(5):
    to_return.append(loc_list[start:start+len_loc])
    start+=len_loc

locationQueries=[]
for row in to_return:
    locationQueries.append(''.join(map(str, [location+" OR " for location in row]))[:-4])

In [ ]:
print("Locations Queries\n\n", locationQueries,"\n")
print("Keywords Query\n\n", keywordsQuery)

Locations Queries

 ['puducherry OR jaipur OR jalandhar OR india OR ahmedabad OR bengaluru OR dehradun OR shimla OR sangrur OR arunachal OR kerala', 'bangalore OR thiruvananthapuram OR jammu OR bangkok OR karnataka OR andhra OR pulwama OR delhi-newdelhi OR agartala OR madhya OR gujarat', 'guwahati OR chandigarh OR imphal OR bhubaneswar OR coimbatore OR assam OR salem OR haryana OR patiala OR punjab OR westbengal', 'hyderabad OR srinagar OR ludhiana OR maharashtra OR kolkata OR uttar OR chennai OR tamil OR patna OR gurgaon OR karnal', 'ranchi OR delhi OR bathinda OR telangana OR amritsar OR madurai OR tiruchirappalli OR lucknow'] 

Keywords Query

 protest OR protests OR riot OR riots OR violence OR unrest OR clash OR bandh OR issue OR rally OR demonstration OR election OR polls OR attac


In [ ]:
print(dt.datetime.now())
print(dt.datetime(2022,4,5,0,0,0))

2022-04-26 10:39:04.578196
2022-04-05 00:00:00


In [ ]:
start_date = dt.datetime(2022,4,1,0,0,0)
end_date = dt.datetime(2022,4,7,0,0,0)

#twitter api
cnt = 0
tweetList = []
for locationQuery in locationQueries:
    tweets = twitter.cursor(twitter.search, q=keywordsQuery + ' (' +locationQuery + ') -filter:retweets', count=100, fromDate=start_date,toDate=end_date, tweet_mode = "extended")
    for item in tweets:
        if (cnt==100):
            break
            print("Progress: "+str(cnt))
        tweetList.append(item)
        cnt+=1
        if cnt > 40000:
            print("Thread sleep")
            #time.sleep(60*15+5)
            cnt = 0
    break
tweetsDF = pd.DataFrame(tweetList)

In [ ]:
tweetsDF

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,lang,possibly_sensitive,extended_entities,quoted_status_id,quoted_status_id_str,quoted_status
0,Tue Apr 26 11:32:02 +0000 2022,1518915805995364352,1518915805995364352,@hombalefilms PLEASE ITS MY HUMBLE REQUEST TO ...,False,"[0, 254]","{'hashtags': [{'text': 'KGFChapter2', 'indices...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,Tue Apr 26 11:31:46 +0000 2022,1518915737855078400,1518915737855078400,India is swiftly building it's muscle against ...,False,"[0, 240]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
2,Tue Apr 26 11:31:41 +0000 2022,1518915715721736194,1518915715721736194,"कोरोना से हो गई थी परिजनों की मौत, मुआवजा लेने...",False,"[0, 133]","{'hashtags': [{'text': 'Haryana', 'indices': [...","{'iso_language_code': 'hi', 'result_type': 're...","<a href=""https://about.twitter.com/products/tw...",NaN,...,0,0,False,False,hi,False,NaN,NaN,NaN,NaN
3,Tue Apr 26 11:31:37 +0000 2022,1518915697786580994,1518915697786580994,@DerekJGrossman 3. And total failure to unders...,False,"[16, 293]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.518645e+18,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
4,Tue Apr 26 11:31:16 +0000 2022,1518915612780675074,1518915612780675074,@LloydGB1962 @robertsessford1 @PaulBrandITV @P...,False,"[65, 274]","{'hashtags': [{'text': 'durhampartygate', 'ind...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/#!/download/ipad"" ...",1.518893e+18,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Tue Apr 26 11:00:41 +0000 2022,1518907912936210433,1518907912936210433,Kishor has been engaged by Mamata Banerjee's T...,False,"[0, 249]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,1,10,False,False,en,False,NaN,NaN,NaN,NaN
96,Tue Apr 26 11:00:18 +0000 2022,1518907817239130112,1518907817239130112,@anurag_hota You are looking at this purely fr...,False,"[13, 213]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",1.518907e+18,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
97,Tue Apr 26 11:00:00 +0000 2022,1518907743066857472,1518907743066857472,#BQOpinion | The #LICIPO should not be a ‘tick...,False,"[0, 264]","{'hashtags': [{'text': 'BQOpinion', 'indices':...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://about.twitter.com/products/tw...",NaN,...,1,1,False,False,en,False,NaN,NaN,NaN,NaN
98,Tue Apr 26 10:59:50 +0000 2022,1518907701849632768,1518907701849632768,@whirlpoolusa @whirlpool_india @WhirlpoolCare ...,False,"[14, 254]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.518898e+18,...,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Querying twitter API
#twitter api

date = dt.datetime.now()
cnt = 0
tweetList = []
for locationQuery in locationQueries:
    tweets = twitter.cursor(twitter.search, q=keywordsQuery + ' (' +locationQuery + ') -filter:retweets', count=100, tweet_mode = "extended")
    for item in tweets:
        if (cnt%10000 == 0):
            print("Progress: "+str(cnt))
        tweetList.append(item)
        cnt+=1
        if cnt > 40000:
            print("Thread sleep")
            time.sleep(60*15+5)
            cnt = 0
tweetsDF = pd.DataFrame(tweetList)

Progress: 0


KeyboardInterrupt: ignored

In [ ]:
# Lookup to get full text for each tweet (first fetch gets only truncated text)
#twitter api
cnt = 1
rowInd = 1
tempList = []
tweetList = []
tweetsDfs = []
tweets = tweetsDF
for index, row in tweets.iterrows():
    tempList.append(row["id"])
    if cnt%100 == 0:
        try:
            tweetList.extend(twitter.lookup_status(id=tempList, tweet_mode = "extended"))
            tempList = []
        except:
            print("Succesful till: "+cnt)
    if (cnt%10000 == 0):
        print("Progress: "+str(cnt))
        if cnt >= 40000:
            tweetsDfs.append(pd.DataFrame(tweetList))
#             tweetsDf.to_csv("Tweets3Weeks"+str(date.day)+str(date.strftime("%m"))+"_"+str(rowInd)+".csv")
            tweetList = []
            rowInd+=1
#             print("Thread sleep")
#             time.sleep(60*15+5)
            cnt = 0
    cnt+=1
tweetList.extend(twitter.lookup_status(id=tempList, tweet_mode = "extended"))
tweetsDfs.append(pd.DataFrame(tweetList))

TypeError: ignored

In [ ]:
tweetsDF = pd.concat(tweetsDfs)

In [ ]:
tweetsDF.shape
tweetsDF.index = np.arange(len(tweetsDF))
date = dt.datetime.now()
tweetsDF.to_csv("tweets_collected_"+str(date.day)+str(date.strftime("%m"))+".csv")

Pre-processing tweets to get useful features

In [ ]:
def get_features(row):
    row["userlocation"] = row["user"]["location"]
    row["username"] = row["user"]["name"]
    row["userscreen_name"] = row["user"]["screen_name"]
    row["userdescription"] = row["user"]["description"]
    row["userfollowers_count"] = row["user"]["followers_count"]
    row["userfriends_count"] = row["user"]["friends_count"]
    row["userlisted_count"] = row["user"]["listed_count"]
    row["userfavourites_count"] = row["user"]["favourites_count"]
    row["userverified"] = row["user"]["verified"]
    row["userstatuses_count"] = row["user"]["statuses_count"]
    row["userfollowing"] = row["user"]["following"]
    row["userfollow_request_sent"] = row["user"]["follow_request_sent"]
    row["usercontributors_enabled"] = row["user"]["contributors_enabled"]
    sepr =  ""
    l = []
    for location in locations['Locations']:
        if (location in str(row["full_text"]).lower()):
            l.append(location)
    row["extracted_location"] = ','.join(l)
    return row

In [ ]:
processed_data = tweetsDF.apply(get_features, axis=1)
processed_data.index = np.arange(len(processed_data))

In [ ]:
# Map tweets to acled locations

acledLocations = ['Jammu','Srinagar','Delhi-New Delhi','Bathinda','Dehradun','Chandigarh','Ludhiana','Amritsar',
                  'Imphal','Lucknow','Jalandhar','Kolkata','Guwahati','Chennai','Patiala','Bhubaneswar','Bengaluru',
                  'Patna','Jaipur','Coimbatore','Hyderabad','Ranchi','Thiruvananthapuram','Shimla','Sangrur',
                  'Ahmedabad','Karnal','Pulwama','Puducherry','Gurgaon','Agartala','Madurai','Tiruchirappalli',
                  'Bangkok','Salem','India']

def generate_locations(row):
    dfrow_list = []
    for location in acledLocations:
        tempLocation = location
        tempLocation2 = location
        if (tempLocation == "Delhi-New Delhi"):
            tempLocation = "delhi"
        elif (tempLocation == "Bengaluru"):
            tempLocation = "bangalore"
            tempLocation2 = "karnataka"
        elif (tempLocation == "Kolkata"):
            tempLocation = "west bengal"
        elif (tempLocation == "Guwahati"):
            tempLocation = "assam"
        elif (tempLocation in ("Lucknow", "Dehradun")):
            tempLocation = "uttar"
        elif (tempLocation in ("Sangrur", "Ludhiana", "Amritsar", "Chandigarh", "Bathinda", "Jalandhar", "Patiala")):
            tempLocation = "punjab"
        elif (tempLocation in ("Karnal", "Gurgaon", "Chandigarh")):
            tempLocation2 = "haryana"
        elif (tempLocation == "Pulwama"):
            tempLocation2 = "jammu"
        elif (tempLocation == "Ahmedabad"):
            tempLocation = "gujarat"
        elif (tempLocation in ("Chennai", "Coimbatore", "Madurai", "Tiruchirappalli", "Salem")):
            tempLocation = "tamil"
        elif (tempLocation in ("Imphal", "Agartala")):
            tempLocation = "arunachal"
        elif (tempLocation == "Hyderabad"):
            tempLocation = "andhra"
            tempLocation2 = "telangana"
        elif (tempLocation == "Thiruvananthapuram"):
            tempLocation = "kerala"
        if (tempLocation.lower() in row["extracted_location"] or location.lower() in row["extracted_location"] or tempLocation2 in row["extracted_location"]):
            row["finallocation"] = location
            dfrow_list.append(row.copy())
    return pd.DataFrame(dfrow_list)

In [ ]:
posssed_df = pd.concat(processed_data.apply(generate_locations, axis = 1).tolist())
posssed_df.index = np.arange(len(posssed_df))

In [ ]:
try:
    df = pd.read_json("TweetsConsolidted.json")
    df = pd.concat([posssed_df, df])
    df.index = np.arange(len(df))
except:
    df = posssed_df
df.to_json("TweetsConsolidated.json")

In [ ]:
posssed_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,...,userfriends_count,userlisted_count,userfavourites_count,userverified,userstatuses_count,userfollowing,userfollow_request_sent,usercontributors_enabled,extracted_location,finallocation
0,Tue Apr 26 02:41:56 +0000 2022,1518782400415039488,1518782400415039488,@KannadaNaduu Did you protest same when Bangal...,False,"[14, 137]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://twitter.com/download/android"" ...",1.518455e+18,1518455389922238464,...,272,1,14060,False,3616,None,None,False,bangalore,Bengaluru
1,Tue Apr 26 01:43:23 +0000 2022,1518767666726465537,1518767666726465537,Farmers of Punjab in past had protested agains...,False,"[0, 281]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,958,2,15144,False,2980,None,None,False,punjab,Bathinda
2,Tue Apr 26 01:43:23 +0000 2022,1518767666726465537,1518767666726465537,Farmers of Punjab in past had protested agains...,False,"[0, 281]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,958,2,15144,False,2980,None,None,False,punjab,Chandigarh
3,Tue Apr 26 01:43:23 +0000 2022,1518767666726465537,1518767666726465537,Farmers of Punjab in past had protested agains...,False,"[0, 281]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,958,2,15144,False,2980,None,None,False,punjab,Ludhiana
4,Tue Apr 26 01:43:23 +0000 2022,1518767666726465537,1518767666726465537,Farmers of Punjab in past had protested agains...,False,"[0, 281]","{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,958,2,15144,False,2980,None,None,False,punjab,Amritsar
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93792,Mon Apr 18 10:15:00 +0000 2022,1515997314397077511,1515997314397077511,রাজনৈতিক ক্ষমতা বলে ত্রিপুরায় নারী নির্যাতন অ...,False,"[0, 272]",{'hashtags': [{'text': 'TripuraDeservesBetter'...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,...,0,0,7,False,556,None,None,False,"india,agartala",India
93793,Mon Apr 18 04:40:59 +0000 2022,1515913258615832581,1515913258615832581,Not even one show is housefull today for #Beas...,False,"[0, 150]","{'hashtags': [{'text': 'Beast', 'indices': [41...","<a href=""http://twitter.com/download/android"" ...",NaN,None,...,118,0,1059,False,1464,None,None,False,chennai,Chennai
93794,Mon Apr 18 07:00:00 +0000 2022,1515948240696401920,1515948240696401920,निकाय चुनाव को लेकर भाजपा तैयार : धनखड़\nhttps...,False,"[0, 72]","{'hashtags': [{'text': 'haryana', 'indices': [...","<a href=""https://about.twitter.com/products/tw...",NaN,None,...,388,12,123,False,26174,None,None,False,haryana,Karnal
93795,Mon Apr 18 07:00:00 +0000 2022,1515948240696401920,1515948240696401920,निकाय चुनाव को लेकर भाजपा तैयार : धनखड़\nhttps...,False,"[0, 72]","{'hashtags': [{'text': 'haryana', 'indices': [...","<a href=""https://about.twitter.com/products/tw...",NaN,None,...,388,12,123,False,26174,None,None,False,haryana,Gurgaon


In [ ]:
from searchtweets import collect_results, gen_rule_payload, load_credentials, api_utils
import pandas as pd
import time
import datetime

ModuleNotFoundError: ignored